<a href="https://colab.research.google.com/github/VishaalChandrasekar0203/ChampainFinance_Insights_LLMs/blob/main/Campagin_financing_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [ ]:
!pip install langchain chromadb transformers torch sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 2.2 MB/s eta 0:00:00

In [ ]:
import os
import pandas as pd
from huggingface_hub import login
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
# Set your Hugging Face access token
os.environ["HUGGINGFACE_TOKEN"] = "your hf token"

# Login to Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])


In [ ]:
# Define column names for the dataset
column_names = [
    'COMMITTEE_ID', 'AMENDMENT_INDICATOR', 'REPORT_TYPE', 'ELECTION_CYCLE', 'TRANSACTION_ID',
    'LINE_NUMBER', 'CONTRIBUTOR_TYPE', 'CONTRIBUTOR_NAME', 'CONTRIBUTOR_CITY', 'CONTRIBUTOR_STATE',
    'CONTRIBUTOR_ZIP', 'CONTRIBUTOR_EMPLOYER', 'CONTRIBUTOR_OCCUPATION', 'CONTRIBUTION_DATE',
    'CONTRIBUTION_AMOUNT', 'RECIPIENT_ID', 'TRANSACTION_CODE', 'FILE_NUMBER', 'MEMO_CODE',
    'MEMO_TEXT', 'SUB_ID'
]

In [ ]:
# Load the dataset from the text file
df = pd.read_csv('itcont_2024_20000101_20230330.txt', sep='|', names=column_names, header=None)

<ipython-input-17-f19f723cafa4>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('itcont_2024_20000101_20230330.txt', sep='|', names=column_names, header=None)


In [ ]:
def initialize_chroma(embeddings):
    client = chromadb.Client()
    collection = client.create_collection(
        "campaign_finance",
        embedding_function=embeddings.embed_documents
    )

    batch_size = 1000
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        documents = batch.apply(lambda row: f"Committee ID: {row['COMMITTEE_ID']}, Contributor: {row['CONTRIBUTOR_NAME']}, Amount: ${row['CONTRIBUTION_AMOUNT']}, Date: {row['CONTRIBUTION_DATE']}, Occupation: {row['CONTRIBUTOR_OCCUPATION']}", axis=1).tolist()
        embeddings_list = embeddings.embed_documents(documents)
        collection.add(
            embeddings=embeddings_list,
            documents=documents,
            metadatas=[row.to_dict() for _, row in batch.iterrows()],
            ids=[f"id{i+j}" for j in range(len(documents))]
        )
        print(f"Processed batch {i//batch_size + 1} of {len(df)//batch_size + 1}")

    return collection

In [ ]:
def setup_vector_store():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = Chroma(
        collection_name="campaign_finance",
        embedding_function=embeddings
    )
    return vector_store, embeddings

vector_store, embeddings = setup_vector_store()
chroma_collection = initialize_chroma(embeddings)

UniqueConstraintError: Collection campaign_finance already exists

In [ ]:
def initialize_llm():
    model_name = "meta-llama/Llama-2-7b-chat-hf"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    tokenizer = AutoTokenizer.from_pretrained(model_name, token=os.environ["HUGGINGFACE_TOKEN"])
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        token=os.environ["HUGGINGFACE_TOKEN"],
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    ).to(device)

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15,
        device=device,
    )

    return HuggingFacePipeline(pipeline=pipe)

llm = initialize_llm()

Using device: cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def setup_qa_chain():
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True
    )

qa_chain = setup_qa_chain()

In [ ]:
def answer_question(question):
    result = qa_chain.invoke({"query": question})
    return result["result"], result["source_documents"]

# Example usage
question = "Who made the largest contribution and how much was it?"
answer, sources = answer_question(question)

print(f"Question: {question}")
print(f"Answer: {answer}")
print("Sources:")
for source in sources:
    print(f"- {source.page_content}")

# Additional example of using metadata filtering
large_contributions = vector_store.similarity_search_with_score(
    "Large contributions",
    filter={"CONTRIBUTION_AMOUNT": {"$gt": 1000}},
    k=5
)

print("\nLarge Contributions:")
for doc, score in large_contributions:
    print(f"- {doc.page_content} (Relevance: {score})")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Question: Who made the largest contribution and how much was it?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Who made the largest contribution and how much was it?
Helpful Answer: The total contribution for the fundraising campaign was $150,000. Of this amount, John Smith made the largest contribution of $75,000.
Additional Information:
•	John Smith is a wealthy businessman who has been a long-time supporter of the charity.
•	The fundraising campaign was organized by the charity’s board of directors to raise money for a new building project.
•	Other individuals and organizations also contributed to the campaign, but their contributions were significantly smaller than John Smith’s.
Sources:

Large Contributions:
